In [ ]:
import spacy
import pandas as pd

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define gender swap logic
def swap_gender(sentence, target_gender='female'):
    doc = nlp(sentence)
    new_tokens = []

    if target_gender == 'female':
        swap_dict = {
            "he": "she", "him": "her", "his": "her", "himself": "herself"
        }
    else:
        swap_dict = {
            "she": "he", "her": "him", "hers": "his", "herself": "himself"
        }

    for token in doc:
        word = token.text
        lower_token = word.lower()

        if lower_token == "her":
            if token.dep_ in ('dobj', 'pobj'):
                replacement = 'him' if target_gender == 'male' else 'her'
            elif token.dep_ in ('poss',):
                replacement = 'his' if target_gender == 'male' else 'her'
            else:
                replacement = swap_dict.get(lower_token, word)
        elif lower_token == "his":
            if token.dep_ == 'poss':
                replacement = 'her' if target_gender == 'female' else 'his'
            else:
                replacement = swap_dict.get(lower_token, word)
        elif lower_token in swap_dict:
            replacement = swap_dict[lower_token]
        else:
            replacement = word

        if word[0].isupper():
            replacement = replacement.capitalize()

        new_tokens.append(replacement)

    return ''.join([t + doc[i].whitespace_ for i, t in enumerate(new_tokens)])


# === Load CSV ===
df = pd.read_csv("/content/pronoun_testcases.csv")  # Replace with your file path

# === Apply Function and Compare ===
results = []
mismatches = []

for idx, row in df.iterrows():
    input_text = row['input_text']
    target_gender = row['target_gender'].lower()
    expected = row['expected_output']

    predicted = swap_gender(input_text, target_gender)

    correct = predicted.strip() == expected.strip()
    results.append(correct)

    if not correct:
        mismatches.append({
            "Input": input_text,
            "Target": target_gender,
            "Predicted": predicted,
            "Expected": expected
        })

# === Report Accuracy ===
accuracy = sum(results) / len(results) * 100
print(f"✅ Accuracy: {accuracy:.2f}% ({sum(results)}/{len(results)})\n")

# === Show Mismatches ===
if mismatches:
    print("❌ Mismatches:")
    for m in mismatches:
        print(f"Input: {m['Input']}")
        print(f"Predicted: {m['Predicted']}")
        print(f"Expected : {m['Expected']}")
        print("---")
else:
    print("All outputs matched expected results!")


✅ Accuracy: 100.00% (26/26)

All outputs matched expected results!


In [ ]:
PART3
